In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import os
import foursquare
import requests as re
from IPython.display import JSON # this makes a field expandable return structure of the json information
import numpy as np

The main goal of the mini-project is to build the database of restaurants, bars and various points of interest (POIs) in the area of your choice and find out which API has better coverage in the selected area. The APIs have limited number of requests for free, so start with the smaller area. The project consists of following tasks:

- pull the data about various POIs in the area through API. (Search Yelp for companiees that are in the area using [these instructions](https://www.yelp.com/developers/documentation/v3/business_search)). 
    - If you run out of requests for any of the APIs, don't worry, it's ok to use only sample data you already have or the POIs from the Yelp API. It's approach and process that counts, not the actual number of places we were able to get.
- create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. We've used and created sqlite3 databases before in the activity [**SQL in Python**](https://data.compass.lighthouselabs.ca/b9e08cd5-68c6-490c-a32b-a66f01bf53e1).
- compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.
- choose the top 10 POIs based on the popularity (number of reviews or average rating) ([Yelp](https://www.yelp.com/developers/documentation/v3/business), [Foursquare](https://developer.foursquare.com/docs/api-reference/venues/details/)).
- (Stretch) By implementing [travelling salesman problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem), how much time would it take to visit all of these places? ([Directions API](https://developers.google.com/maps/documentation/directions/start) from google will be helpful here). We will have to find travel time between all places (top 10). We can use [ortools](https://developers.google.com/optimization/routing/tsp) from Google to effectively implement TSP. These tools are very powerful and [easy to install](https://developers.google.com/optimization/install).

# YELP /businesses/search

This endpoint returns up to 1000 businesses based on the provided search criteria. It has some basic information about the business. To get detailed information and reviews, please use the Business ID returned here and refer to [/businesses/{id}](https://www.yelp.com/developers/documentation/v3/business) and [/businesses/{id}/reviews](https://www.yelp.com/developers/documentation/v3/business_reviews) endpoints.

In [28]:
yelp_id = os.environ['yelp_ID']
yelp_key = os.environ['yelp_Key']

location = 'Waterton Park'

url = f'https://api.yelp.com/v3/businesses/search?location={location}&limit=50'
headers = {'Authorization': 'Bearer %s' % yelp_key}

# commented out to avoid running multiple requests
# res = re.get(url, headers=headers)
yelp_json = res.json()

In [28]:
name_businesses_yelp = []
for i in yelp_json['businesses']:
    name_businesses_yelp.append(i['name'])
name_businesses_yelp

['Wieners of Waterton',
 'Waffleton',
 'Trappers Mountain Grill',
 "Zum's Restaurant and Mercantile",
 'Lakeside Chophouse',
 'Pearls Cafe',
 '49 Degrees North Pizza',
 'Pizza of Waterton',
 'The Taco Bar',
 'Wildflower Ave Corner Coffee',
 'Waterton Lakes National Park',
 'Thirsty Bear Waterton - Kitchen & Bar',
 'Red Rock Trattoria',
 'Fireside Lounge & Wine Bar',
 "Zum's Eatery In Waterton",
 "Vimy's Lounge & Grill",
 'The Big Scoop',
 'Waterton Bagel & Coffee',
 'Mountain Top Yogurt Shoppe',
 'Waterton Lakes National Park Alberta',
 'Bertha Lake Trail',
 'BeaverTails',
 'Glacier Bistro',
 'Windsor Lounge',
 'Akamina Parkway',
 'Royal Stewart Dining Room',
 'Subway']

In [29]:
JSON(yelp_json)

<IPython.core.display.JSON object>

In [53]:
# attempting to create a yelp sqlite database
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [54]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

connection = create_connection("yelp.sqlite")

Connection to SQLite DB successful


In [26]:
#create_business_table = """
#CREATE TABLE IF NOT EXISTS businesses (
#  id INTEGER PRIMARY KEY AUTOINCREMENT,
#  name TEXT NOT NULL,
#  business_id INTEGER,
#  review_count INTEGER,
#  rating INTEGER
#  latitude INTEGER
#  longitude INTEGER
#);
#"""

#execute_query(connection, create_business_table)  

Query executed successfully


In [45]:
# grab the information I want into a list of lists, with first index to be column headers later
def yelp_business_details(data):
    yelp_data = [['name','business_id','rating','review_count','latitude','longitude']]
    for i in data['businesses']:
        yelp_data.append([
            i['name'],
            i['id'],
            i['rating'],
            i['review_count'],
            i['coordinates']['latitude'],
            i['coordinates']['longitude']
        ])
    return yelp_data

In [64]:
yelp_df = pd.DataFrame(yelp_business_details(yelp_json))
yelp_df.columns = yelp_df.iloc[0]
yelp_df = yelp_df[1:]

more_1_review = yelp_df['review_count'] > 1
yelp_df[more_1_review].sort_values(by=['rating','review_count'], ascending=False).head(10)

,name,business_id,rating,review_count,latitude,longitude
11,Waterton Lakes National Park,vYVc3QhRcZfthy0A7yMaVA,5.0,24,49.082896,-113.915587
1,Wieners of Waterton,nGkAzQtKPKyN67cyso5wUw,4.5,74,49.052073,-113.907795
2,Waffleton,rJ6row-Kq57UcyOcCl-Qqg,4.5,28,49.052043,-113.907825
7,49 Degrees North Pizza,YzBmc4OJVz-vWViXgqI08g,4.5,11,49.051845,-113.908287
10,Wildflower Ave Corner Coffee,R5Ki1NqnWC3y3U2RsDNLyg,4.5,9,49.050998,-113.908491
13,Red Rock Trattoria,wq-AYDQtwKzkv34bfgm4zw,4.0,15,49.052962,-113.909972
14,Fireside Lounge & Wine Bar,sBWHIK-GnppueEMVMQ-g-w,4.0,12,49.052269,-113.906167
15,Zum's Eatery In Waterton,SfN1FBGMjgXAXa1jFzI3vA,4.0,6,49.04829,-113.90767
3,Trappers Mountain Grill,lzCs5g98NnJtcD0AnWfRWg,3.5,42,49.052087,-113.907002
4,Zum's Restaurant and Mercantile,mnw-CowIWVDPeUddxLIxSg,3.5,30,49.047909,-113.908203


In [19]:
yelp_ll=[
    ['49.082896,-113.915587'],
    ['49.052073%2C-113.907795'],
    ['49.052043%2C-113.907825'],
    ['49.051845%2C-113.908287'],
    ['49.050998%2C-113.908491'],
    ['49.052962%2C-113.909972'],
    ['49.052269%2C-113.906167'],
    ['49.04829%2C-113.90767'],
    ['49.052087%2C-113.907002'],
    ['49.047909,-113.908203']]

In [55]:
# push df to sqlite database, making a table called 'businesses' in the process
# yelp_df.to_sql('businesses', connection) # commented out to avoid pushing again

# Foursquare

### venue/search

In [6]:
foursquare_id = os.environ['FOURSQUARE_CLIENT_ID']
foursquare_key = os.environ['FOURSQUARE_CLIENT_SECRET']

#url = 'https://api.foursquare.com/v2/venues/search'
params = dict(
client_id = foursquare_id,
client_secret = foursquare_key,
v='20180323',
near='Waterton Park, AB',
limit=50
)
# commented out to avoid running multiple requests
#res = re.get(url=url, params=params)
foursq_json = res.json()

In [58]:
JSON(foursq_json)

<IPython.core.display.JSON object>

In [7]:
# grab the information I want into a list of lists, with first index to be column headers later
def fsq_business_details(data):
    foursq_search = [['name','business_id','latitude','longitude']]
    for i in data['response']['venues']:
        foursq_search.append([
            i['name'],
            i['id'],
            i['location']['lat'],
            i['location']['lng']
        ])
    return foursq_search

In [8]:
fsq_search_df = pd.DataFrame(fsq_business_details(foursq_json))
fsq_search_df.columns = fsq_search_df.iloc[0]
fsq_search_df = fsq_search_df[1:]
# make a ll column which we can pull with the business id to run the for loop
fsq_search_df['ll'] = fsq_search_df['latitude'].astype(str) + ',' + fsq_search_df['longitude'].astype(str)
fsq_search_df['rating'] = np.nan # throwing in an empty rating column to sub in later
fsq_search_df.head()

,name,business_id,latitude,longitude,ll,rating
1,Northland Lodge,5d1ea01d1feae30023e1e384,49.050064,-113.91653,"49.050064,-113.91653",NaN
2,Cameron Falls,4e5975dc18a8c204750cd4bf,49.051341,-113.91604,"49.051340829924605,-113.9160397415404",NaN
3,"Waterton, Alberta",4e3d79e07d8b0e961069ce63,49.052142,-113.908082,"49.05214241377369,-113.90808152342375",NaN
4,Bertha Trailhead,503127fce4b08362164cc687,49.04531,-113.916983,"49.045309697785044,-113.91698320006687",NaN
5,Zums Eatery,4c7d28479221236a4f4f7e3d,49.049293,-113.907389,"49.04929322706642,-113.90738904697935",NaN


In [67]:
# create the foursq search business list database
connection = create_connection("foursq_search.sqlite")
# push df to sqlite database, making a table called 'businesses' in the process
# fsq_search_df.to_sql('businesses', connection) # comment out to avoid pushing again

Connection to SQLite DB successful


### venue/explore

In [73]:
# comparing this explore endpoint against search to see if results differ
url = 'https://api.foursquare.com/v2/venues/explore'
params = dict(
client_id = foursquare_id,
client_secret = foursquare_key,
v='20180323',
near='Waterton Park, AB',
limit=50
)
# commented out to avoid running multiple requests
# res = re.get(url=url, params=params)
foursq2_json = res.json()
JSON(foursq2_json)

<IPython.core.display.JSON object>

In [77]:
# grab the information I want into a list of lists, with first index to be column headers later
def fsq_explore_business(data):
    foursq_explore = [['name','business_id','latitude','longitude']]
    for i in data['response']['groups'][0]['items']:
        foursq_explore.append([
            i['venue']['name'],
            i['venue']['id'],
            i['venue']['location']['lat'],
            i['venue']['location']['lng']
        ])
    return foursq_explore

In [122]:
fsq_explore_df = pd.DataFrame(fsq_explore_business(foursq2_json)) # get that dataframe
fsq_explore_df.columns = fsq_explore_df.iloc[0] # make those headers
fsq_explore_df = fsq_explore_df[1:] # don't repeat those headers
# make a ll column which we can pull with the business id to run the for loop
fsq_explore_df['ll'] = fsq_explore_df['latitude'].astype(str) + ',' + fsq_explore_df['longitude'].astype(str)
fsq_explore_df['rating'] = np.nan # throwing in an empty rating column to sub in later

fsq_explore_df.head()

,name,business_id,latitude,longitude,ll,rating
1,Waterton Lakes National Park,4c4600b3dd35be9a888edcd8,49.052595,-113.907303,"49.052595120014175,-113.90730311169038",NaN
2,Wieners Of Waterton,4c929f27418ea1cd3e8ca685,49.052092,-113.907914,"49.05209231850583,-113.90791448646839",NaN
3,Prince Of Wales Hotel,4c2ceb6c2219c92840e5a648,49.058295,-113.903918,"49.058294929906424,-113.90391754319757",NaN
4,Bayshore Inn,4bc9fc90a276eee173386202,49.05241,-113.906445,"49.05241049451198,-113.90644507395069",NaN
5,Trappers Mountain Grill,4e092b05d4c03ae0b9d47bf2,49.0521,-113.907097,"49.052099837192216,-113.90709653694292",NaN


In [79]:
# create the foursq search business list database
connection = create_connection("foursq_explore.sqlite")
# push df to sqlite database, making a table called 'businesses' in the process
fsq_explore_df.to_sql('businesses', connection) # comment out to avoid pushing again

Connection to SQLite DB successful


### Venue ID ratings

In [131]:
# TEST on one business first to understand the response
#url = 'https://api.foursquare.com/v2/venues/4e87b20dbe7bcd2a2aaf356f	' # pearl's cafe
params = dict(
client_id = foursquare_id,
client_secret = foursquare_key,
v='20180323',
)
# commented out to avoid running multiple requests
# res = re.get(url=url, params=params)
fsqglacier_json = res.json()
JSON(fsqglacier_json)

<IPython.core.display.JSON object>

In [124]:
# running the for loop for all businesses found via venue EXPLORE
# don't touch this -- already run and ranked
#for i in range(1,len(fsq_explore_df)):
    venue_id = fsq_explore_df.loc[i,'business_id']
    
    url = f'https://api.foursquare.com/v2/venues/{venue_id}'
    params = dict(
    client_id = foursquare_id,
    client_secret = foursquare_key,
    v='20180323',
    )
    
    # commented out to avoid running multiple requests
    #res = re.get(url=url, params=params)
    if res.status_code == 200: # this helps us stop naturally is quota met or connection fails 
        fsqE_json = res.json()
        try:
            fsq_explore_df.loc[i,'rating'] = fsqE_json['response']['venue']['rating']
        except:
            pass # not all venues have a rating so we don't want to error out part way


In [9]:
# running the for loop for all businesses found via venue SEARCH
#for i in range(1,len(fsq_search_df)):
    venue_id = fsq_search_df.loc[i,'business_id']
    
    url = f'https://api.foursquare.com/v2/venues/{venue_id}'
    params = dict(
    client_id = foursquare_id,
    client_secret = foursquare_key,
    v='20180323',
    )
    
    # commented out to avoid running multiple requests
    #res = re.get(url=url, params=params)
    if res.status_code == 200: # this helps us stop naturally is quota met or connection fails 
        fsqS_json = res.json()
        try:
            fsq_search_df.loc[i,'rating'] = fsqS_json['response']['venue']['rating']
        except:
            pass # not all venues have a rating so we don't want to error out part way


In [125]:
#fsq_explore_df # don't touch this, already ranked and run

,name,business_id,latitude,longitude,ll,rating
1,Waterton Lakes National Park,4c4600b3dd35be9a888edcd8,49.052595,-113.907303,"49.052595120014175,-113.90730311169038",8.5
2,Wieners Of Waterton,4c929f27418ea1cd3e8ca685,49.052092,-113.907914,"49.05209231850583,-113.90791448646839",7.6
3,Prince Of Wales Hotel,4c2ceb6c2219c92840e5a648,49.058295,-113.903918,"49.058294929906424,-113.90391754319757",7.6
4,Bayshore Inn,4bc9fc90a276eee173386202,49.05241,-113.906445,"49.05241049451198,-113.90644507395069",7.3
5,Trappers Mountain Grill,4e092b05d4c03ae0b9d47bf2,49.0521,-113.907097,"49.052099837192216,-113.90709653694292",7.2
6,Cameron Falls,4e5975dc18a8c204750cd4bf,49.051341,-113.91604,"49.051340829924605,-113.9160397415404",6.9
7,Waterton Glacier Suites,4e4444f8aeb7d02fd0e915b0,49.053057,-113.909945,"49.05305654087697,-113.9099450531108",NaN
8,Red Rock Canyon,520a8e1b498e62ca3523019b,49.051323,-113.881763,"49.051323168841286,-113.88176344494849",NaN
9,Waterton Lakes Lodge,4bc8eaf9762beee13eb23d38,49.051616,-113.90928,"49.05161636,-113.90928",NaN
10,Pizza of Waterton,4c07e7ea7e3fc928a174ef82,49.0517,-113.908529,"49.051699593336814,-113.90852902804002",NaN


In [12]:
#fsq_search_df.sort_values('rating',ascending=False) #dont touch this

,name,business_id,latitude,longitude,ll,rating
2,Cameron Falls,4e5975dc18a8c204750cd4bf,49.051341,-113.91604,"49.051340829924605,-113.9160397415404",6.9
12,Subway,5006fe5ce4b07575b865ff81,49.052048,-113.908235,"49.0520484263999,-113.90823543534783",6.4
1,Northland Lodge,5d1ea01d1feae30023e1e384,49.050064,-113.91653,"49.050064,-113.91653",NaN
3,"Waterton, Alberta",4e3d79e07d8b0e961069ce63,49.052142,-113.908082,"49.05214241377369,-113.90808152342375",NaN
4,Bertha Trailhead,503127fce4b08362164cc687,49.04531,-113.916983,"49.045309697785044,-113.91698320006687",NaN
5,Zums Eatery,4c7d28479221236a4f4f7e3d,49.049293,-113.907389,"49.04929322706642,-113.90738904697935",NaN
6,The Church of Jesus Christ of Latter-day Saints,4d9b8460c175dcb3a583d07b,49.052017,-113.910413,"49.0520171264152,-113.91041278839111",NaN
7,Thirsty Bear Waterton - Kitchen & Bar,5d1192878132b900231e8d08,49.052684,-113.906257,"49.0526836,-113.9062574",NaN
8,Red Rock Canyon,4e87b3ff722e45a6b8a2f19c,49.05278,-113.912993,"49.05278027180712,-113.91299328897817",NaN
9,The Church of Jesus Christ of Latter-day Saints,4f731f847bebc1de47d22d93,49.099464,-113.907187,"49.099464087238495,-113.90718729608393",NaN


- (Stretch) By implementing [travelling salesman problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem), how much time would it take to visit all of these places? ([Directions API](https://developers.google.com/maps/documentation/directions/start) from google will be helpful here). We will have to find travel time between all places (top 10). We can use [ortools](https://developers.google.com/optimization/routing/tsp) from Google to effectively implement TSP. These tools are very powerful and [easy to install](https://developers.google.com/optimization/install).

# Google Directions API

In [10]:
api_key = os.environ['directions_Key']
#url = 

In [22]:
yelp_ll
waypoints = yelp_ll[1:9]
waypoints

[['49.052073%2C-113.907795'],
 ['49.052043%2C-113.907825'],
 ['49.051845%2C-113.908287'],
 ['49.050998%2C-113.908491'],
 ['49.052962%2C-113.909972'],
 ['49.052269%2C-113.906167'],
 ['49.04829%2C-113.90767'],
 ['49.052087%2C-113.907002']]

In [28]:
# Directions API - YELP
url = f'https://maps.googleapis.com/maps/api/directions/json?origin={yelp_ll[0]}&destination={yelp_ll[9]}&waypoints=optimize:true%7Cvia:{waypoints[0]}%7Cvia:{waypoints[1]}%7Cvia:{waypoints[2]}%7Cvia:{waypoints[3]}%7Cvia:{waypoints[4]}%7Cvia:{waypoints[5]}%7Cvia:{waypoints[6]}%7Cvia:{waypoints[7]}&mode=walking&key={api_key}'
res = re.get(url)
yelp_directions_json = res.json()
JSON(yelp_directions_json)

<IPython.core.display.JSON object>

# Google ORTools

In [6]:
"""Simple travelling salesman problem between cities."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 7293 miles
Route for vehicle 0:
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 -> 9 -> 0



In [3]:
# Create the data - Google ORTools
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [4]:
# Create the routing model - Google ORToots
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

NameError: name 'pywrapcp' is not defined

In [5]:
# Create the distance callback
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
      

NameError: name 'routing' is not defined

In [ ]:
# Set the cost of travel
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [ ]:
# Set search parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [ ]:
# Add the solution printer
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [ ]:
# Solve and print the solution
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

In [ ]:
# Run the programs


In [ ]:
# Save routes to a list or array
def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

In [ ]:
routes = get_routes(solution, routing, manager)
# Display the routes.
for i, route in enumerate(routes):
  print('Route', i, route)